In [1]:
pip install openai-agents nest_asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from agents import Agent, InputGuardrail,GuardrailFunctionOutput, Runner
from pydantic import BaseModel
import asyncio

import nest_asyncio
nest_asyncio.apply()

class HomeworkOutput(BaseModel):
  is_homework: bool
  reasoning: str

guardrail_agent = Agent(
  name="Guardrail check",
  instructions="Check if the user is asking about homework.",
  output_type=HomeworkOutput,
)

math_tutor_agent = Agent(
  name="Math Tutor",
  handoff_description="Specialist agent for math questions",
  instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

history_tutor_agent = Agent(
  name="History Tutor",
  handoff_description="Specialist agent for historical questions",
  instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

async def homework_guardrail(ctx, agent, input_data):
  result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
  final_output = result.final_output_as(HomeworkOutput)
  return GuardrailFunctionOutput(
    output_info=final_output,
    tripwire_triggered=not final_output.is_homework,
  )

triage_agent = Agent(
  name="Triage Agent",
  instructions="You determine which agent to use based on the user's homework question",
  handoffs=[history_tutor_agent, math_tutor_agent],
  input_guardrails=[
    InputGuardrail(guardrail_function=homework_guardrail),
  ],
)

async def main():
  result = await Runner.run(triage_agent, "who was the first president of the united states?")
  print(result.final_output)

if __name__ == "__main__":
  asyncio.run(main())

The first president of the United States was George Washington. He served from 1789 to 1797. Washington was unanimously elected by the Electoral College and is often called the "Father of His Country" due to his pivotal role in the founding of the United States. He set many precedents for the presidency and was instrumental in the drafting and ratification of the U.S. Constitution. Washington served two terms and voluntarily stepped down, reinforcing the principle of peaceful transition of power.


### Basic configuration
The most common properties of an agent you'll configure are:

- instructions: also known as a developer message or system prompt.
- model: which LLM to use, and optional model_settings to configure model tuning parameters like temperature, top_p, etc.
- tools: Tools that the agent can use to achieve its tasks.

In [7]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
  return f"The weather in {city} is sunny"

agent = Agent(
  name="Haiku agent",
  instructions="Always respond in haiku form",
  tools=[get_weather],
)

result = Runner.run_sync(agent, "what is the weather in New York?")
print(result.final_output)

Skies are clear and bright,  
In New York, the sun shines strong,  
A perfect day dawns.


### Context

Agents are generic on their context type. Context is a dependency-injection tool: it's an object you create and pass to Runner.run(), that is passed to every agent, tool, handoff etc, and it serves as a grab bag of dependencies and state for the agent run. You can provide any Python object as the context.

In [13]:
from agents import Agent, Runner
from dataclasses import dataclass

@dataclass
class UserContext:
  name: str
  uid: str
  is_pro_user: bool

  async def fetch_purchases(self) -> list[str]:
    return ['PS5', 'Xbox Series X']

agent = Agent[UserContext](
  name="Pro User Agent",
  instructions="Provide special responses to pro users",
)

### Output types

By default, agents produce plain text (i.e. str) outputs. If you want the agent to produce a particular type of output, you can use the output_type parameter. A common choice is to use Pydantic objects, but we support any type that can be wrapped in a Pydantic TypeAdapter - dataclasses, lists, TypedDict, etc.

In [10]:
from pydantic import BaseModel
from agents import Agent


class CalendarEvent(BaseModel):
  name: str
  date: str
  participants: list[str]

agent = Agent(
  name="Calendar extractor",
  instructions="Extract calendar events from text",
  output_type=CalendarEvent,
)

result = Runner.run_sync(agent, "Meeting with John at 3pm on Friday")
print(result.final_output)

name='Meeting with John' date='This Friday at 3pm' participants=['John']


### Handoffs

Handoffs are sub-agents that the agent can delegate to. You provide a list of handoffs, and the agent can choose to delegate to them if relevant. This is a powerful pattern that allows orchestrating modular, specialized agents that excel at a single task. Read more in the handoffs documentation.

In [ ]:
from agents import Agent

booking_agent = Agent(
  name="Booking agent",
  instructions="Help the user with booking queries",
)
refund_agent = Agent(
  name="Refund agent",
  instructions="Help the user with refund queries",
)

triage_agent = Agent(
  name="Triage agent",
  instructions=(
    "Help the user with their questions."
    "If they ask about booking, handoff to the booking agent."
    "If they ask about refunds, handoff to the refund agent."
  ),
  handoffs=[booking_agent, refund_agent],
)

### Dynamic instructions

In most cases, you can provide instructions when you create the agent. However, you can also provide dynamic instructions via a function. The function will receive the agent and context, and must return the prompt. Both regular and async functions are accepted.

In [29]:
from agents import Agent, RunContextWrapper, Runner

def dynamic_instructions(
  context: RunContextWrapper[UserContext], agent: Agent[UserContext]
) -> str:
  list_of_purchases = context.context.fetch_purchases()
  return f"User Pro:{context.context.is_pro_user}\nList of purchases: {list_of_purchases}\nYou help pro users only! If the user is not pro, tell them to look their purchases up in their account."

@dataclass
class UserContext:
  name: str
  uid: str
  is_pro_user: bool

  def fetch_purchases(self) -> list[str]:
    return ['PS5', 'Xbox Series X']

agent = Agent[UserContext](
  name="Pro User Agent",
  instructions=dynamic_instructions,
)

result = Runner.run_sync(agent, "What are my purchases?", context=UserContext(name='dog', uid="123", is_pro_user=True))
print(result.final_output)
result = Runner.run_sync(agent, "What are my purchases?", context=UserContext(name='dog', uid="123", is_pro_user=False))
print(result.final_output)


Your purchases include a PS5 and an Xbox Series X.
Please check your account to view your purchases.


### Lifecycle events (hooks)

Sometimes, you want to observe the lifecycle of an agent. For example, you may want to log events, or pre-fetch data when certain events occur. You can hook into the agent lifecycle with the hooks property. Subclass the [AgentHooks](https://openai.github.io/openai-agents-python/ref/lifecycle/#agents.lifecycle.AgentHooks) class, and override the methods you're interested in.

### Guardrails

Guardrails allow you to run checks/validations on user input, in parallel to the agent running. For example, you could screen the user's input for relevance. Read more in the [guardrails](https://openai.github.io/openai-agents-python/guardrails/) documentation.

Guardrails run in parallel to your agents, enabling you to do checks and validations of user input. For example, imagine you have an agent that uses a very smart (and hence slow/expensive) model to help with customer requests. You wouldn't want malicious users to ask the model to help them with their math homework. So, you can run a guardrail with a fast/cheap model. If the guardrail detects malicious usage, it can immediately raise an error, which stops the expensive model from running and saves you time/money.

There are two kinds of guardrails:

1. Input guardrails run on the initial user input
2. Output guardrails run on the final agent output

#### Input guardrails

Input guardrails run in 3 steps:

1. First, the guardrail receives the same input passed to the agent.
2. Next, the guardrail function runs to produce a GuardrailFunctionOutput, which is then wrapped in an InputGuardrailResult
3. Finally, we check if .tripwire_triggered is true. If true, an InputGuardrailTripwireTriggered exception is raised, so you can appropriately respond to the user or handle the exception.

#### Output guardrails

Output guardrails run in 3 steps:

1. First, the guardrail receives the same input passed to the agent.
2. Next, the guardrail function runs to produce a GuardrailFunctionOutput, which is then wrapped in an OutputGuardrailResult
3. Finally, we check if .tripwire_triggered is true. If true, an OutputGuardrailTripwireTriggered exception is raised, so you can appropriately respond to the user or handle the exception.

In [ ]:
from pydantic import BaseModel
from agents import (
  Agent,
  GuardrailFunctionOutput,
  InputGuardrailTripwireTriggered,
  RunContextWrapper,
  Runner,
  TResponseInputItem,
  input_guardrail,
)

class MathHomeworkOutput(BaseModel):
  is_math_homework: bool
  reasoning: str

guardrail_agent = Agent(
  name="Guardrail check",
  instructions="Check if the user is asking you to do their math homework.",
  output_type=MathHomeworkOutput,
)


@input_guardrail
async def math_guardrail(
  ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
  result = await Runner.run(guardrail_agent, input, context=ctx.context)

  return GuardrailFunctionOutput(
    output_info=result.final_output,
    tripwire_triggered=result.final_output.is_math_homework,
  )


agent = Agent(
  name="Customer support agent",
  instructions="You are a customer support agent. You help customers with their questions.",
  input_guardrails=[math_guardrail],
)

async def main():
  # This should trip the guardrail
  try:
    await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
    print("Guardrail didn't trip - this is unexpected")

  except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")

In [ ]:
from pydantic import BaseModel
from agents import (
  Agent,
  GuardrailFunctionOutput,
  OutputGuardrailTripwireTriggered,
  RunContextWrapper,
  Runner,
  output_guardrail,
)
class MessageOutput(BaseModel):
  response: str

class MathOutput(BaseModel):
  is_math: bool
  reasoning: str

guardrail_agent = Agent(
  name="Guardrail check",
  instructions="Check if the output includes any math.",
  output_type=MathOutput,
)

@output_guardrail
async def math_guardrail(
    ctx: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
  result = await Runner.run(guardrail_agent, output.response, context=ctx.context)

  return GuardrailFunctionOutput(
    output_info=result.final_output,
    tripwire_triggered=result.final_output.is_math,
  )

agent = Agent(
  name="Customer support agent",
  instructions="You are a customer support agent. You help customers with their questions.",
  output_guardrails=[math_guardrail],
  output_type=MessageOutput,
)

async def main():
  # This should trip the guardrail
  try:
    await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
    print("Guardrail didn't trip - this is unexpected")

  except OutputGuardrailTripwireTriggered:
    print("Math output guardrail tripped")